# Safe String Lookup

Current heavylight.Table functionality (1.0.5) doesn't carry out validation of table keys before use, i.e. it uses the Band functionality.

This works fine as long as the lookup values are all in the keys (and the keys are sorted in order), however it fails where keys don't exist.

This notebook explores adding a np.isin() test before doing the lookup.

This is likely to be relatively expensive, as the key check only really needs to happen at the start (string keys are from data)

In [1]:
import numpy as np
import numba as nb

In [2]:
table_keys = np.array(['A', 'B', 'C', 'D'], dtype='<U2')
table_vals = np.array(['AA', 'BB', 'CC', 'DD'], dtype='<U2')


In [3]:
xs = np.array(['B', 'C', 'A', 'AB', 'D']) # 'AB' should return an error
table_vals[np.searchsorted(table_keys, xs)]

array(['BB', 'CC', 'AA', 'BB', 'DD'], dtype='<U2')

Here, 'AB' should return an error, however because 'A' < 'AB' < 'B', searchsorted identifies it as being at index B.

In [4]:
np.isin(xs, table_keys)

array([ True,  True,  True, False,  True])

In [5]:
rng = np.random.default_rng(seed=42)
big_xs = rng.choice(xs, 100_000)
big_xs

array(['B', 'AB', 'AB', ..., 'A', 'B', 'D'], dtype='<U2')

## Alternative: numba exact equality testing

In [6]:
@nb.njit(nogil=True, parallel=True)
def find_exact(keys:np.ndarray, key_array:np.ndarray) -> np.ndarray:
    ret_arr = -9999 * np.ones(keys.shape[0], dtype=np.int32)   # start with sentinel
    for i in nb.prange(keys.shape[0]):
        for j in nb.prange(key_array.shape[0]):
            if keys[i] == key_array[j]:
                ret_arr[i] = j
                # break ## break is commented out as using parallelisation
    return ret_arr

In [7]:
find_exact(xs, table_keys)

array([    1,     2,     0, -9999,     3])

In [8]:
def find_exact_py(keys:np.ndarray, key_array:np.ndarray) -> np.ndarray:
    ret_arr = -9999 * np.ones(keys.shape[0], dtype=np.int32)   # start with sentinel
    for i in range(keys.shape[0]):
        for j in range(key_array.shape[0]):
            if keys[i] == key_array[j]:
                ret_arr[i] = j
                break
    return ret_arr

In [9]:
find_exact_py(xs, table_keys)

array([    1,     2,     0, -9999,     3], dtype=int32)

In [10]:
%timeit find_exact(big_xs, table_keys)

8.19 ms ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit find_exact_py(big_xs, table_keys)

82 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
def find_exact_ss(keys:np.ndarray, key_array:np.ndarray) -> np.ndarray:
    return np.where(np.isin(keys, key_array), 
                    np.searchsorted(key_array, keys),
                    -9999)

In [13]:
find_exact_ss(xs, table_keys)

array([    1,     2,     0, -9999,     3])

In [14]:
%timeit find_exact_ss(big_xs, table_keys)

2.65 ms ± 33.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
def find_exact_unsafe(keys: np.ndarray, key_array:np.ndarray) -> np.ndarray:
    return np.searchsorted(key_array, keys)


In [16]:
%timeit find_exact_unsafe(big_xs, table_keys)

1.48 ms ± 8.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
find_exact(big_xs, table_keys)

array([    1, -9999, -9999, ...,     0,     1,     3])

In [18]:
nb.types.UnicodeCharSeq(2)

UnicodeCharSeq(2)

In [19]:
# nb.types.Array(nb.types.UnicodeCharSeq(2))

In [20]:
find_exact.inspect_types()

find_exact (Array(UnicodeCharSeq(2), 1, 'C', False, aligned=True), Array(UnicodeCharSeq(2), 1, 'C', False, aligned=True))
--------------------------------------------------------------------------------
# File: /var/folders/n4/gpw_j7653_l052l8phfj8lbr0000gn/T/ipykernel_48347/3325116556.py
# --- LINE 1 --- 
# label 0
#   keys = arg(0, name=keys)  :: array([unichr x 2], 1d, C)
#   keys_shape.0 = getattr(value=keys, attr=shape)  :: UniTuple(int64 x 1)
#   keys_size0.1 = static_getitem(value=keys_shape.0, index=0, index_var=None, fn=<built-in function getitem>)  :: int64
#   del keys_shape.0
#   key_array = arg(1, name=key_array)  :: array([unichr x 2], 1d, C)

@nb.njit(nogil=True, parallel=True)

# --- LINE 2 --- 

def find_exact(keys:np.ndarray, key_array:np.ndarray) -> np.ndarray:

    # --- LINE 3 --- 
    #   id=0[LoopNest(index_variable = parfor_index.9, range = (0, keys_size0.1, 1))]{315: <ir.Block at /var/folders/n4/gpw_j7653_l052l8phfj8lbr0000gn/T/ipykernel_48347/3325116556.py (3)

In [21]:


@nb.njit(nogil=True, parallel=True)
def find_exact_typed(keys:nb.types.UnicodeCharSeq(2)[:], key_array:nb.types.UnicodeCharSeq(2)[:]) -> np.ndarray:
    ret_arr = -9999 * np.ones(keys.shape[0], dtype=np.int32)   # start with sentinel
    for i in nb.prange(keys.shape[0]):
        for j in nb.prange(key_array.shape[0]):
            if keys[i] == key_array[j]:
                ret_arr[i] = j
                # break ## break is commented out as using parallelisation
    return ret_arr

In [22]:
nb.types.UnicodeCharSeq(2)[:]

Array(UnicodeCharSeq(2), 1, 'A', False, aligned=True)

In [23]:
find_exact_typed(big_xs, table_keys)

array([    1, -9999, -9999, ...,     0,     1,     3])

In [24]:
%timeit find_exact_typed(big_xs, table_keys)

8.28 ms ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit find_exact(big_xs, table_keys)

8.29 ms ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Very little difference (I may not have configured it correctly)

In [31]:
table_keys[np.searchsorted(table_keys, big_xs)] == big_xs

array([ True, False, False, ...,  True,  True,  True])

In [27]:
big_xs

array(['B', 'AB', 'AB', ..., 'A', 'B', 'D'], dtype='<U2')

In [28]:
table_keys

array(['A', 'B', 'C', 'D'], dtype='<U2')